# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 210MB of 14542MB


# Import Customized modules and Create ImageJ Object

In [3]:
import os
import sys
import re
from typing import TextIO
from datetime import datetime

from glob import glob
import logging
from skimage.io import imread, imshow, imsave


sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module

from logger import init_logger
from fileop import create_new_dir
from composite_ij import dump_info, median_R1_and_mean3D_R2

In [4]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] some ImageJ plugins need to be new before use
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
from ij.plugin import ChannelSplitter
from ij.plugin import ZProjector
from ij.plugin import RGBStackMerge
from ij.plugin import RGBStackConverter

rm = RoiManager()
imageCalculator = ImageCalculator()
channelSplitter = ChannelSplitter()
zProjector = ZProjector()
rgbStackMerge = RGBStackMerge()
rgbStackConverter = RGBStackConverter()

In [5]:
log = init_logger(r"ImageJ")

# Start Process

In [6]:
# lif_path = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\palmskin_RGB_RAW\Before_20221109\20220610 CE001 palmskin_8dpf.lif"
lif_path = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\BrightField_RAW\Before_20221109\20220610 CE001 palmskin_8dpf.lif"

Reader.setId(lif_path)
seriesCount = Reader.getSeriesCount()
log.info(f'seriesCount : {seriesCount}')

| 2023-03-03 07:22:52,785 | ImageJ | INFO | seriesCount : 6


In [7]:
SeriesNum = 1

ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>

In [8]:
dump_info(img)
img.show()
# img.hide()

    name  : 20220610 CE001 palmskin_8dpf.lif - Series001 fish 1 palmskin 8dpf
    type  : <java class 'ij.ImagePlus'>
    dtype : N/A
    shape : (2048, 2048)
    dims  : ('X', 'Y')



RGB 

In [10]:
image_name = img.getProp("Image name")
log.info(f'image_name {type(image_name)}: {image_name}')


z_length = img.getNumericProperty("Image #0|DimensionDescription #6|Length")
z_slice = img.getNumericProperty("Image #0|DimensionDescription #6|NumberOfElements")
log.info(f'z_length {type(z_length)}: {z_length}')
log.info(f'z_slice {type(z_slice)}: {z_slice}')

voxel_z = z_length/(z_slice-1)*(10**6)
log.info(f'voxel_z {type(voxel_z)}: {voxel_z:.4f} micron/pixel')

| 2023-03-02 14:49:54,815 | ImageJ | INFO | image_name <java class 'java.lang.String'>: Series001 fish 1 palmskin_8dpf_A
| 2023-03-02 14:49:54,815 | ImageJ | INFO | z_length <java class 'JDouble'>: 7.19568e-05
| 2023-03-02 14:49:54,816 | ImageJ | INFO | z_slice <java class 'JDouble'>: 37.0
| 2023-03-02 14:49:54,816 | ImageJ | INFO | voxel_z <class 'float'>: 1.9988 micron


BF

In [10]:
image_name = img.getProp("Image name")
log.info(f'image_name {type(image_name)}: {image_name}')


dim1_length = img.getNumericProperty("Image #0|DimensionDescription #1|Length")
dim1_elem = img.getNumericProperty("Image #0|DimensionDescription #1|NumberOfElements")
log.info(f'dim1_length {type(dim1_length)}: {dim1_length}')
log.info(f'dim1_elem {type(dim1_elem)}: {dim1_elem}')

dim2_length = img.getNumericProperty("Image #0|DimensionDescription #2|Length")
dim2_elem = img.getNumericProperty("Image #0|DimensionDescription #2|NumberOfElements")
log.info(f'dim2_length {type(dim2_length)}: {dim2_length}')
log.info(f'dim2_elem {type(dim2_elem)}: {dim2_elem}')

dim1_unit = dim1_length/(dim1_elem-1)*(10**6)
dim2_unit = dim2_length/(dim2_elem-1)*(10**6)

assert dim1_unit == dim2_unit, f"Voxel_X != Voxel_Y, Voxel_X, Voxel_Y = ({dim1_unit}, {dim2_unit}) micron/pixel"

log.info(f'Voxel_X_Y {type(dim1_unit)}: {dim1_unit}')

| 2023-03-03 07:23:09,670 | ImageJ | INFO | image_name <java class 'java.lang.String'>: Series001 fish 1 palmskin 8dpf
| 2023-03-03 07:23:09,671 | ImageJ | INFO | dim1_length <java class 'JDouble'>: 0.00665275
| 2023-03-03 07:23:09,672 | ImageJ | INFO | dim1_elem <java class 'JDouble'>: 2048.0
| 2023-03-03 07:23:09,674 | ImageJ | INFO | dim2_length <java class 'JDouble'>: 0.00665275
| 2023-03-03 07:23:09,674 | ImageJ | INFO | dim2_elem <java class 'JDouble'>: 2048.0
| 2023-03-03 07:23:09,675 | ImageJ | INFO | Voxel_X_Y <class 'float'>: 3.25
